In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# 1. Carregar o dataset e manter apenas a primeira coluna
dataset = pd.read_csv('heart_rate.csv')
heartbeats = dataset.iloc[:, 0].values  # Mantendo apenas a primeira coluna

# 2. Função para criar sequências de dados para o treinamento do modelo
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# 3. Definindo o tamanho das sequências
seq_length = 10

# 4. Criando sequências de dados
X, y = create_sequences(heartbeats, seq_length)

# 5. Dividindo os dados em treinamento (70%) e teste (30%)
split_index = int(0.7 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# 6. Remodelando os dados para o formato esperado pelo RNN
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# 7. Criando o modelo RNN
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(1))

# 8. Compilando o modelo
model.compile(optimizer='adam', loss='mse')

# 9. Treinando o modelo
model.fit(X_train, y_train, epochs=200, validation_split=0.2, verbose=1)

# 10. Fazendo previsões com o modelo treinado
predictions = model.predict(X_test)

# 11. Arredondando as previsões para o inteiro mais próximo
predictions = np.round(predictions).astype(int)

# Exibindo algumas previsões
print("Previsões dos próximos batimentos cardíacos (inteiros):", predictions.flatten())

# 12. Utilizando 15 novos batimentos para prever os próximos 10 batimentos
new_heartbeats = np.array([89, 90, 91, 91, 92, 91, 91, 90, 92, 93, 88, 89, 90, 91, 92])
next_heartbeats = []

# Prever os próximos 10 batimentos
for i in range(10):
    input_seq = new_heartbeats[-seq_length:].reshape((1, seq_length, 1))
    next_heartbeat = model.predict(input_seq)
    next_heartbeat = np.round(next_heartbeat).astype(int)[0][0]
    next_heartbeats.append(next_heartbeat)
    new_heartbeats = np.append(new_heartbeats, next_heartbeat)

print("Próximos 10 batimentos cardíacos previstos (inteiros):", next_heartbeats)

# 13. Calculando a média dos próximos 10 batimentos previstos
mean_predicted = np.round(np.mean(next_heartbeats)).astype(int)
print("Média dos próximos 10 batimentos cardíacos previstos (inteiros):", mean_predicted)

# 14. Inserindo manualmente 10 novos batimentos cardíacos
manual_heartbeats = np.array([95, 99, 103, 105, 107, 107, 111, 114, 114, 113])
mean_manual = np.round(np.mean(manual_heartbeats)).astype(int)
print("Média dos 10 novos batimentos cardíacos inseridos manualmente (inteiros):", mean_manual)

# 15. Comparando as médias e emitindo alertas
if mean_predicted > mean_manual + 5:
    print("Alerta: O batimento cardíaco subiu muito em relação ao previsto.")
elif mean_predicted < mean_manual - 5:
    print("Aviso: O batimento cardíaco caiu muito em relação ao previsto.")
else:
    print("Os batimentos cardíacos estão dentro da faixa esperada.")


Epoch 1/200
32/32 [==============================] - 1s 11ms/step - loss: 1429.0272 - val_loss: 10.5804
Epoch 2/200
32/32 [==============================] - 0s 5ms/step - loss: 18.2133 - val_loss: 11.3940
Epoch 3/200
32/32 [==============================] - 0s 5ms/step - loss: 5.6687 - val_loss: 10.7106
Epoch 4/200
32/32 [==============================] - 0s 4ms/step - loss: 5.3658 - val_loss: 10.4162
Epoch 5/200
32/32 [==============================] - 0s 4ms/step - loss: 5.2700 - val_loss: 10.4272
Epoch 6/200
32/32 [==============================] - 0s 4ms/step - loss: 5.2987 - val_loss: 10.3887
Epoch 7/200
32/32 [==============================] - 0s 4ms/step - loss: 5.3002 - val_loss: 10.3731
Epoch 8/200
32/32 [==============================] - 0s 4ms/step - loss: 5.3943 - val_loss: 10.3199
Epoch 9/200
32/32 [==============================] - 0s 4ms/step - loss: 5.3229 - val_loss: 10.2825
Epoch 10/200
32/32 [==============================] - 0s 6ms/step - loss: 5.3300 - val_loss: 10